In [1]:
%matplotlib inline
import itertools
from time import time
import csv
import dask
from dask.diagnostics import ProgressBar

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from kifmm_py import KiFmm

In [2]:
from dask.distributed import LocalCluster
client = LocalCluster().get_client()

2024-06-08 13:48:12,070 - distributed.dashboard.components.shared - ERROR - 'NoneType' object has no attribute 'add_next_tick_callback'
Traceback (most recent call last):
  File "/Users/sri/Code/kifmm/.venv/lib/python3.10/site-packages/distributed/utils.py", line 832, in wrapper
    return await func(*args, **kwargs)
  File "/Users/sri/Code/kifmm/.venv/lib/python3.10/site-packages/distributed/dashboard/components/shared.py", line 315, in cb
    self.doc().add_next_tick_callback(lambda: self.update(prof, metadata))
AttributeError: 'NoneType' object has no attribute 'add_next_tick_callback'
2024-06-08 13:48:12,077 - tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x134836170>>, <Task finished name='Task-41656' coro=<ProfileTimePlot.trigger_update.<locals>.cb() done, defined at /Users/sri/Code/kifmm/.venv/lib/python3.10/site-packages/distributed/utils.py:830> exception=

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60061,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:60072,Total threads: 2
Dashboard: http://127.0.0.1:60076/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:60064,


In [4]:
def setup_fmm_blas(sources, targets, charges, kernel_eval_type, kernel, field_translation, surface_diff, svd_threshold, depth, expansion_order):
    tmp = [expansion_order] * (depth + 1)

    start_time = time()
    fmm = KiFmm(
        tmp,
        sources,
        targets,
        charges,
        kernel_eval_type,
        kernel,
        field_translation,
        prune_empty=True,
        timed=True,
        svd_threshold=svd_threshold,
        surface_diff=surface_diff,
        depth=depth
    )
    setup_time = time() - start_time
    return fmm, setup_time

In [5]:
np.random.seed(0)
surface_diff_vec = [0, 1, 2]
svd_threshold_vec = [None, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 2e-2]
depth_vec = [4, 5]
expansion_order_vec = [3, 4, 5, 6]

parameters = list(itertools.product(surface_diff_vec, svd_threshold_vec, depth_vec, expansion_order_vec))

dim = 3
dtype = np.float32
ctype = np.complex64

# Set FMM Parameters
n_vec = 1
n_crit = None
depth = 3
n_sources = 1000000
n_targets = 1000000
kernel = "laplace"
field_translation = "blas"
kernel_eval_type = (
    "eval"
)

sources = np.reshape(
    np.random.rand(n_sources * dim), (n_sources, dim)
).astype(dtype)
targets = np.reshape(
    np.random.rand(n_targets * dim), (n_targets, dim)
).astype(dtype)
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec)
).astype(dtype)

rel_error = []
times = []
setup = []

fmms = []
for (i, (surface_diff, svd_threshold, depth, expansion_order)) in enumerate(parameters):
    fmm = client.submit(
        setup_fmm_blas,
        sources,
        targets,
        charges,
        kernel_eval_type,
        kernel,
        field_translation,
        surface_diff,
        svd_threshold,
        depth,
        expansion_order
    )
    fmms.append(fmm)

/Users/sri/Code/kifmm/.venv/lib/python3.10/site-packages/distributed/client.py:3164: UserWarning: Sending large graph of size 26.70 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
fmms = client.gather(fmms)